# Visualizing The Triangle

## Imports

In [ ]:
import csv
import json
import math
import numpy as np
import pandas as pd
import plotly.express as px
import tempfile

from pathlib import Path

## Read the data and build derived columns

In [ ]:
### Fix smart quotes and ingest data

In [ ]:
text=Path("Triangle_Atlas.csv").read_text()
text.replace("\u2019","'")
with tempfile.TemporaryDirectory() as td:
    csv_file=(Path(td) / "this.csv")
    csv_file.write_text(text)
    df=pd.read_csv(csv_file)

### Tweak it so it's more useful to us

In [ ]:
# replace NaN with string 'None' (for labels)
df.fillna(value='None', inplace=True)

df['Y']= -df['Y']  # They're all south
# So dot notation works
df.rename(columns={"Import Index": "Imports", "Export Index": "Exports"}, inplace=True)

### Create derived columns

In [ ]:
# Symbol index:

# UFP: blue circle
# Klingon: red square
# Klingon client: darker red open square
# Romulan: green cross
# Independent: brownish open diamond
# Others: yellowish diamond

symbols={
    "Affiliation of Outer Worlds": {"color": "orange",
                                    "marker": "diamond"},
    "Imperial Klingon States": {"color": "firebrick",
                                "marker": "square-open"},
    "Independent Worlds": {"color": "darkgoldenrod",
                           "marker": "diamond-open"},
    "Klingon Empire": { "color": "red",
                       "marker": "square"},
    "Mantiev Colonial Association": {"color": "yellow",
                                     "marker": "diamond"},
    "Orion Frontier Mercantile Association": {"color": "yellowgreen",
                                              "marker": "diamond"},
    "Romulan Star Empire": {"color": "green",
                            "marker": "cross"},
    "UFP": {"color": "blue",
            "marker": "circle"}
}

ms = []
cs = []
an = []
for r in df.itertuples(index=False):
    k=r.Affiliation
    i=r.Imports
    e=r.Exports
    ms.append(symbols[k]['marker'])
    cs.append(symbols[k]['color'])
    anno = f"{r.Name}: {r.Affiliation}\n"
    anno += f"Imports: {i}\n"
    anno += f"Exports: {e}"
    an.append(anno)

### Append the columns to the right edge of the dataframe

In [ ]:
df.insert(len(df.columns), 'Marker', ms)
df.insert(len(df.columns), 'Color', cs)
df.insert(len(df.columns), 'Annotation', an)

## Construct distance map

In [ ]:
dist_dict: dict[str,dict[str,float]] = {}

locs=df.get(["Name", "X", "Y", "Z"])
recs=locs.to_records(column_dtypes={"X": "float", "Y": "float", "Z": "float"})
max_recnum=len(recs) - 1
for rec in recs:
    idx=rec[0]
    src_name=rec[1]
    src_x=rec[2]
    src_y=rec[3]
    src_z=rec[4]
    if src_name not in dist_dict:
        dist_dict[src_name] = {}
    dist_dict[src_name][src_name] = 0
    if idx < max_recnum:
        for cmp_idx in range(idx+1,len(df)):
            cmp_rec = recs[cmp_idx]
            cmp_name=cmp_rec[1]
            dst_x=cmp_rec[2]
            dst_y=cmp_rec[3]
            dst_z=cmp_rec[4]
            dx = src_x - dst_x
            dy = src_y - dst_y
            dz = src_z - dst_z
            distance = math.sqrt( dx*dx + dy*dy + dz*dz )
            if cmp_name not in dist_dict:
                dist_dict[cmp_name] = {}
            dist_dict[src_name][cmp_name] = distance
            dist_dict[cmp_name][src_name] = distance

## Convert dict to list for CSV Export

### Build square array with room for labels

In [ ]:
dist_list: list[list[str|float]] = []
names=sorted(list(dist_dict.keys()))
for i in range(len(names) + 1):
    dist_list.append([])
for i in range(len(names) + 1):
    for j in range(len(names) + 1):
        dist_list[i].append(-1.0)  
dist_list[0][0]="Distance"

### Fill in data

In [ ]:
for idx, n in enumerate(names):
    # Row and Column label is system name
    dist_list[0][idx+1] = n
    dist_list[idx+1][0] = n
    dist_list[idx+1][idx+1] = 0  # Diagonal is zero
    for inner_idx in range(idx+1,len(names)):
        inner_n=names[inner_idx]
        distance = dist_dict[n][inner_n]
        trunc_dist="{0:.2f}".format(distance)
        dist_list[idx+1][inner_idx+1] = trunc_dist
        dist_list[inner_idx+1][idx+1] = trunc_dist

### Write output CSV

In [ ]:
with open("Distances.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(dist_list)

## Plot the data

In [ ]:
fig = px.scatter_3d(data_frame=df, x='X', y='Y', z='Z',
                    color='Color', color_discrete_map='identity',
                    symbol='Marker',
                    symbol_map='identity',
                    title='The Triangle<br><sup>Coordinate Origin: Sol System</sup>',
                    hover_data=['Name','Affiliation','Imports','Exports','X','Y','Z'],
                    width=1200, height=800
                   )

# Add the axis titles and set the aspect mode to reflect the range
fig.update_layout(scene = dict(
                  xaxis_title='pc (spinward)',
                  yaxis_title='pc (coreward)',
                  zaxis_title='pc (galactic axis)',
                  aspectmode='data'))
fig.show()
